### Imports

In [0]:
import keras # Importing Keras
from keras import backend as K # Importing Keras backend (by default it is Tensorflow)
from keras.datasets import fashion_mnist # Import the fashion-mnist dataset
from keras.layers import Input, Conv2D, Dense, Dropout, Flatten, MaxPool2D, BatchNormalization # Layers to be used for building our model
from keras.models import Model # The class used to create a model
from keras.optimizers import Adam
from keras.utils import np_utils # Utilities to manipulate numpy arrays
from tensorflow import set_random_seed # Used for reproducible experiments
from keras.preprocessing.image import ImageDataGenerator # Used for image dataset handling

import gc
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### Data manipulation

**Task**: Given an image decide the type of clothing

- The data are Grayscale images of size 28x28
- We will use a CNN to classify them
- The values of the inputs are in [0, 255] so we normalize them to [0, 1]

In [0]:
batch_size = 128
classes = 10
epochs = 100

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
        

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print(input_shape)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)


### Plotting results

In [0]:
def plot_history(hs, epochs, metric):
    plt.clf()
    plt.rcParams['figure.figsize'] = [10, 5]
    plt.rcParams['font.size'] = 16
    for label in hs:
        plt.plot(hs[label].history[metric], label='{0:s} train {1:s}'.format(label, metric))
        plt.plot(hs[label].history['val_{0:s}'.format(metric)], label='{0:s} validation {1:s}'.format(label, metric))
    x_ticks = np.arange(0, epochs + 1)
    x_ticks [0] += 1
    plt.xticks(x_ticks)
    plt.ylim((0, 1))
    plt.xlabel('Epochs')
    plt.ylabel('Loss' if metric=='loss' else 'Accuracy')
    plt.legend()
    plt.show()


### Cleaning up

In [0]:
def clean_up(model):
    K.clear_session()
    del model
    gc.collect()


### Model creation

In [0]:
def train_model(
        optimizer,
        epochs=100,
        batch_size=128,
        conv_layers=2,
        conv_activation='relu',
        conv_dropout=False,
        output_activation='softmax'):
    
    np.random.seed(1402) # Define the seed for numpy to have reproducible experiments.
    set_random_seed(1981) # Define the seed for Tensorflow to have reproducible experiments.
    
    # Define the input layer.
    input = Input(
        shape=input_shape,
        name='Input'
    )

    x = input
    
    # Define the convolutional layers.
    x = BatchNormalization()(x)
    x = Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        dilation_rate=(1, 1),
        activation=conv_activation,
        name='Conv2D-1.1'
    )(x)
    x = Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        dilation_rate=(1, 1),
        activation=conv_activation,
        name='Conv2D-1.2'
    )(x)
    x = MaxPool2D(
        pool_size=(2, 2),
        strides=(2, 2),
        padding='same',
        name='MaxPool2D-1'
    )(x)
    x = Dropout(
        rate=0.1,
        name='Dropout-1'
    )(x)
    x = BatchNormalization()(x)
    x = Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        dilation_rate=(1, 1),
        activation=conv_activation,
        name='Conv2D-2.1'
    )(x)
    x = Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        dilation_rate=(1, 1),
        activation=conv_activation,
        name='Conv2D-2.2'
    )(x)
    x = MaxPool2D(
        pool_size=(2, 2),
        strides=(2, 2),
        padding='same',
        name='MaxPool2D-2'
    )(x)

    x = Dropout(
        rate=0.4,
        name='Dropout-2'
    )(x)
    
    x = BatchNormalization()(x)
    x = Conv2D(
      filters=128,
      kernel_size=(3, 3),
      strides=(1, 1),
      padding='same',
      dilation_rate=(1, 1),
      activation=conv_activation,
      name='Conv2D-3.1'
    )(x)
    x = Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding='same',
        dilation_rate=(1, 1),
        activation=conv_activation,
        name='Conv2D-3.2'
    )(x)
    x = MaxPool2D(
        pool_size=(2, 2),
        strides=(2, 2),
        padding='same',
        name='MaxPool2D-3'
    )(x)
    x = Dropout(
        rate=0.5,
        name='Dropout-3'
    )(x)
    
    # Flatten the convolved images so as to input them to a Dense Layer
    x = Flatten(name='Flatten')(x)
    x = Dense(
          units=256,
          activation='relu',
          name='Dense-1'
    )(x)
    
    x = BatchNormalization()(x)
    
    # Define the output layer.
    output = Dense(
        units=classes,
        activation=output_activation,
        name='Output'
    )(x)

     # Define the model and train it.
    model = Model(inputs=input, outputs=output)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    hs = model.fit( 
      x=x_train,
      y=y_train,
      validation_split=0.1, # use 10% of the training data as validation data
      epochs=epochs,
      verbose=1,
      batch_size=batch_size
    )
    
    print('Finished training.')
    print('------------------')
    model.summary() # Print a description of the model.
    return model, hs


Our model initialization

In [0]:
# Using Adam
optimizer = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

# 3 Convolutional Layers
conv_no_drop_model_adam, conv_no_drop_hs_adam = train_model(
    optimizer=optimizer,
    epochs=epochs,
    batch_size=batch_size,
    conv_layers=3,
    conv_activation='relu',
    conv_dropout=False,
    output_activation='softmax'
)

# Evaluate on test data and show all the results.
conv_no_drop_eval_adam = conv_no_drop_model_adam.evaluate(
    x_test,
    y_test,
    verbose=1
)

clean_up(model=conv_no_drop_model_adam)

### Results

In [0]:
print("Train Loss     : {0:.5f}".format(conv_no_drop_hs_adam.history['loss'][-1]))
print("Validation Loss: {0:.5f}".format(conv_no_drop_hs_adam.history['val_loss'][-1]))
print("Test Loss      : {0:.5f}".format(conv_no_drop_eval_adam[0]))
print("---")
print("Train Accuracy     : {0:.5f}".format(conv_no_drop_hs_adam.history['acc'][-1]))
print("Validation Accuracy: {0:.5f}".format(conv_no_drop_hs_adam.history['val_acc'][-1]))
print("Test Accuracy      : {0:.5f}".format(conv_no_drop_eval_adam[1]))


# Plot train and validation error per epoch.
plot_history(hs={'Conv': conv_no_drop_hs_adam}, epochs=epochs, metric='loss')
plot_history(hs={'Conv': conv_no_drop_hs_adam}, epochs=epochs, metric='acc')

In [0]:
exit()